In [3]:
import subprocess
import argparse
import sys
import gzip
import cPickle

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np

class Classifier(object):
    def __init__(self):
        pass

    def train(self):
        """
        Override this method in your class to implement train
        """
        raise NotImplementedError("Train method not implemented")

    def inference(self):
        """
        Override this method in your class to implement inference
        """
        raise NotImplementedError("Inference method not implemented")



def conlleval(p, g, w, filename='tempfile.txt'):
    '''
    INPUT:
    p :: predictions
    g :: groundtruth
    w :: corresponding words

    OUTPUT:
    filename :: name of the file where the predictions
    are written. it will be the input of conlleval.pl script
    for computing the performance in terms of precision
    recall and f1 score
    '''
    out = ''
    for sl, sp, sw in zip(g, p, w):
        out += 'BOS O O\n'
        for wl, wp, ww in zip(sl, sp, sw):
            out += ww + ' ' + wl + ' ' + wp + '\n'
        out += 'EOS O O\n\n'

    f = open(filename, 'w')
    f.writelines(out)
    f.close()

    return get_perf(filename)

def get_perf(filename):
    ''' run conlleval.pl perl script to obtain precision/recall and F1 score '''
    _conlleval = 'conlleval.pl'

    proc = subprocess.Popen(["perl", _conlleval], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, _ = proc.communicate(open(filename).read())
    for line in stdout.split('\n'):
        if 'accuracy' in line:
            out = line.split()
            break

    precision = float(out[6][:-2])
    recall    = float(out[8][:-2])
    f1score   = float(out[10])

    return (precision, recall, f1score)

In [4]:
def make_target(label, size):
    # print "label is ", label
    tensor = torch.zeros(size)
    tensor = tensor.long()
    tensor[label] = 1
    # print "tensor is ", tensor
    return tensor.view(1,-1)
    # return torch.LongTensor([label.tolist()])


class MyNNClassifier(Classifier):
    def __init__(self):
        pass

    def train(self):
        pass

    def inference(self):
        pass

In [144]:
class NeuralNet(nn.Module):  # inheriting from nn.Module!

    def __init__(self, num_input_nodes, num_hidden_nodes, output_dimension):
        super(NeuralNet, self).__init__()
        self.input_linear = nn.Linear(num_input_nodes, num_hidden_nodes)
        self.output_linear = nn.Linear(num_hidden_nodes, output_dimension)

    def forward(self, input_vector):
        out = self.input_linear(input_vector)
        out = F.sigmoid(out)
        out = self.output_linear(out)
        return out

In [145]:
def get_input_vector(word_embeddings, prev_label, word):
    word_feature = autograd.Variable(word_embeddings[word])
    prev_label = autograd.Variable(prev_label)
    input_vector = torch.cat((word_feature.view(1,-1), prev_label.view(1,-1)), 1)
    return input_vector

In [146]:
# argparser = argparse.ArgumentParser()
# argparser.add_argument("--data", type=str, default="atis.small.pkl.gz", help="The zipped dataset")

# parsed_args = argparser.parse_args(sys.argv[1:])

filename = "atis.small.pkl.gz"
f = gzip.open(filename,'rb')
train_set, valid_set, test_set, dicts = cPickle.load(f)

# print "train_set ", train_set

train_lex, _, train_y = train_set
valid_lex, _, valid_y = valid_set
test_lex,  _,  test_y  = test_set

# print "train_lex ", train_lex
# print "train_y ", train_y

idx2label = dict((k,v) for v,k in dicts['labels2idx'].iteritems())
idx2word  = dict((k,v) for v,k in dicts['words2idx'].iteritems())

'''
To have a look what the original data look like, commnet them before your submission
'''
print "length train data ", len(train_lex), " ", len(train_y)
# print "test lex ", test_lex[0]
# print "word dictionary is ", idx2word
# print "label dictionary is ", idx2label
# print train_lex[0], map(lambda t: idx2word[t], train_lex[0])
# print train_y[0], map(lambda t: idx2label[t], train_y[0])
# print test_lex[0], map(lambda t: idx2word[t], test_lex[0])
# print test_y[0], map(lambda t: idx2label[t], test_y[0])

'''
implement you training loop here
'''
NUM_LABELS = len(idx2label)
VOCAB_SIZE = len(idx2word)
HIDDEN_NODES = 50
word_embeddings = torch.rand(VOCAB_SIZE, 100)
# word_embeddings = torch.eye(VOCAB_SIZE, VOCAB_SIZE)
# tag_embeddings = torch.rand(NUM_LABELS+1, 100)
tag_embeddings = torch.eye(NUM_LABELS+1, NUM_LABELS+1)
NUM_INPUT_NODES = len(word_embeddings[0]) + len(tag_embeddings[0])
# print "number of input nodes ", NUM_INPUT_NODES
# print "word_embeddings ", word_embeddings
# print "tag_embeddings ", tag_embeddings
# input dimension for neural network is concatenation of word and tag tensors
model = NeuralNet(NUM_INPUT_NODES, HIDDEN_NODES, NUM_LABELS)

# loss_function = nn.MSELoss(size_average=False)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.03)
# optimizer = optim.Adam(model.parameters(), lr=0.03)

for epoch in range(30):
    epoch_loss = 0
    for sentence, labels in zip(train_lex, train_y):
#         flag = bool(random.getrandbits(1))
#         if flag:
#             continue
        prev_label = tag_embeddings[NUM_LABELS]
        for word, label in zip(sentence, labels):
            model.zero_grad()
            word_feature = autograd.Variable(word_embeddings[word])
            prev_label = autograd.Variable(prev_label)
            input_vector = torch.cat((word_feature.view(1,-1), prev_label.view(1,-1)), 1)

            prev_label = tag_embeddings[label]
            # input_vector = autograd.Variable(concat_vec)
            # print "input vector ", input_vector
            label_tensor = torch.LongTensor([label.item()])
            target = autograd.Variable(label_tensor)
#             target = autograd.Variable(make_target(label, NUM_LABELS))
            probs = model(input_vector)
#             print "probs ", log_probs
#             print "target ", target
            loss = loss_function(probs, target)
            epoch_loss += loss.data[0]
#             print "loss ", loss 
            loss.backward()
            optimizer.step()
    print "epoch number ", epoch, " epoch_loss ", epoch_loss

length train data  3983   3983
epoch number  0  epoch_loss  47435.4185664
epoch number  1  epoch_loss  25380.8490354
epoch number  2  epoch_loss  19819.5599031


KeyboardInterrupt: 

In [147]:
def greedy_inference(model, sentence, word_embeddings, tag_embeddings, NUM_LABELS):
    output_labels = np.zeros(len(sentence))
    prev_label = tag_embeddings[NUM_LABELS]
    for i, word in enumerate(sentence):
        input_vector =  get_input_vector(word_embeddings, prev_label, word)
        probs = model(input_vector)
#         print "probs greedy", probs
        max_val, predicted_label = torch.max(probs, 1)
        predicted_label = predicted_label.data[0]
        prev_label = tag_embeddings[predicted_label]
        output_labels[i] = predicted_label
    return output_labels

In [178]:
def viterbi_inference(model, sentence, word_embeddings, tag_embeddings, NUM_LABELS):
        dp = np.zeros((NUM_LABELS, len(sentence)+1))
        back_pointers = np.zeros((NUM_LABELS, len(sentence)))
        dp[0][0] = 1
        for i in range(len(sentence)):
            word_table = np.zeros((NUM_LABELS, NUM_LABELS))
            if i == 0:
                input_vector = get_input_vector(word_embeddings, tag_embeddings[NUM_LABELS], sentence[i])
#                 print "input_vector ", input_vector
                probs = model(input_vector)
                probs = probs.data.numpy()
#                 print "probs is ", probs
#                 word_table[:,0] = np.log(dp[0, 0]) + np.log(probs)
#                 word_table[:,0] = dp[0,0] + probs
                word_table[:,0] = np.multiply(dp[0, 0], probs)
#                 print "word table 0", word_table[:, 0]
                dp[:,i+1] = word_table[:,0]
                back_pointers[:,i] = 128
                continue
#             print i
            for j in range(NUM_LABELS):
                input_vector = get_input_vector(word_embeddings, tag_embeddings[j], sentence[i])
                probs = model(input_vector)
                probs = probs.data.numpy()
#                 print " probs ", probs
#                 print "dp array ", dp[:,i]
#                 print np.multiply(dp[:, i], probs)
#                 word_table[:,j] = dp[:, i] + probs
                word_table[:,j] = np.multiply(dp[:, i], probs)
#             print "word table is ", word_table
            dp[:,i+1] = word_table.max(1)
            for k in range(NUM_LABELS):
                for index, element in enumerate(word_table[k]):
                    if element == dp[k, i+1]:
                        back_pointers[k, i] = index
#                 back_pointers[k, i] = word_table[k].index(dp[k,i+1])

        print "back_pointers ", back_pointers
        print "dp matrix ", dp[:,19]
        output_labels = np.zeros(len(sentence), dtype = np.int)
        label_index = len(sentence) - 1
        max_val = dp[:, len(sentence)].max()
        for index, element in enumerate(dp[:, len(sentence)]):
            if element == max_val:
                output_labels[label_index] = index
#                 print "debug2 ", index
                break
#         print "output labels ", output_labels
        for i in range(len(sentence)-1, 1, -1):
#             print "debug ", output_labels[label_index]
            row = back_pointers[output_labels[label_index], i]
            label_index -= 1
            output_labels[label_index] = row
        print "output labels ", output_labels
        return output_labels
            

In [179]:
predictions_test = [ map(lambda t: idx2label[t], 
                             viterbi_inference(model, test_lex[0], word_embeddings, tag_embeddings, NUM_LABELS)) 
#                         for x in test_lex
                   ]


back_pointers  [[ 128.  126.   36. ...,  126.   36.  126.]
 [ 128.  126.  126. ...,  126.  126.  126.]
 [ 128.  126.   28. ...,  126.  126.  126.]
 ..., 
 [ 128.   78.   78. ...,   78.   78.   78.]
 [ 128.   28.   28. ...,   28.   28.   28.]
 [ 128.  126.  126. ...,   48.   25.  126.]]
dp matrix  [  5.33354826e+03   1.75822610e+01   2.46865226e+04   5.25872537e+06
   1.33917362e+01   3.91859930e-13   1.56943541e-04   4.14568267e+09
   1.80025229e+04  -9.70991614e+01   1.12501932e+01  -7.21254889e+01
  -1.17978092e-02   3.02004650e+01   5.44606324e+01   1.75830789e+03
  -5.77050083e+07   2.47452169e+04   3.86244346e+07  -7.37616163e+07
   2.75292836e+05   1.88388199e+05  -4.56946790e-05  -7.68558760e+04
  -4.06167878e+04   4.03042204e+02  -2.27209046e+00   2.87254588e+15
   1.39207326e+02   1.52847964e+03   4.48426765e-05   1.29719958e-04
   3.59028893e+02   9.88042419e+04   3.86442967e-06   5.19323715e+04
   3.24249823e+01   6.69112087e-01  -1.56717676e-03   9.52215977e+08
  -6.0918319

In [176]:
print "predictions ", predictions_test[0]
groundtruth_test = [ map(lambda t: idx2label[t], y) for y in test_y ]

predictions  ['B-aircraft_code', 'B-airline_name', 'I-airline_name', 'O', 'B-airline_name', 'I-airline_name', 'O', 'B-airline_name', 'I-airline_name', 'O', 'B-airline_name', 'I-airline_name', 'O', 'O', 'O', 'O', 'B-depart_date.date_relative', 'O', 'O']


In [177]:
print "groundtruth ", groundtruth_test[0]
words_test = [ map(lambda t: idx2word[t], w) for w in test_lex ]
test_precision, test_recall, test_f1score = conlleval(predictions_test, groundtruth_test, words_test)

groundtruth  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name', 'O', 'O', 'O', 'O', 'O', 'B-stoploc.city_name', 'I-stoploc.city_name']


In [157]:
print test_precision, test_recall, test_f1score

60.62 56.12 58.28
